<a href="https://colab.research.google.com/github/kapamawi/AI/blob/main/2_8_3___rekomendacje.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
!pip install -qU langchain_community crewai crewai_tools

Kod instaluje kilka pakietów Pythona używając menedżera pakietów pip:

- langchain_community to biblioteka zawierająca komponenty społecznościowe do budowania aplikacji z użyciem modeli językowych
- crewai to framework do tworzenia zespołów agentów AI, które mogą współpracować przy wykonywaniu zadań
- crewai_tools to zestaw narzędzi rozszerzających możliwości crewai

Flaga -q oznacza "quiet mode" - instalacja będzie przebiegać bez wyświetlania szczegółowych komunikatów.

Flaga -U wymusza aktualizację pakietów do najnowszych dostępnych wersji, nawet jeśli są już zainstalowane w systemie.

Ten jeden wiersz przygotowuje środowisko do pracy z agentami AI opartymi na dużych modelach językowych.

In [ ]:
# Standard library imports
import os
import warnings

# Third-party imports
from crewai import Agent, Task, Crew
from crewai.process import Process
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool
from dotenv import load_dotenv, find_dotenv
from langchain_community.llms import Ollama
from langchain_community.utilities import ApifyWrapper
from langchain_core.documents import Document
from langchain_openai import ChatOpenAI

from google.colab import userdata

import json
# Suppress warnings
warnings.filterwarnings('ignore')


from pprint import pp as ppr


Ten kod importuje szereg bibliotek i narzędzi niezbędnych do pracy z AI i przetwarzania danych:

Ze standardowej biblioteki Pythona:
- os - do operacji systemowych
- warnings - do zarządzania ostrzeżeniami (które są tutaj wyłączone)

Z zewnętrznych bibliotek:
- crewai - importuje główne komponenty do tworzenia zespołów AI:
  - Agent - do tworzenia pojedynczych agentów AI
  - Task - do definiowania zadań
  - Crew - do łączenia agentów w zespoły
  - Process - do zarządzania procesami

- crewai_tools dostarcza specjalistyczne narzędzia:
  - SerperDevTool - do wyszukiwania w internecie
  - ScrapeWebsiteTool - do pobierania danych ze stron
  - WebsiteSearchTool - do przeszukiwania stron

Dodatkowo:
- dotenv - do ładowania zmiennych środowiskowych
- langchain_community.llms.Ollama - do integracji z modelem Ollama
- langchain_openai.ChatOpenAI - do komunikacji z OpenAI
- google.colab.userdata - do obsługi danych użytkownika w Google Colab
- json - do pracy z formatem JSON
- pprint (jako ppr) - do ładnego formatowania wydruków

Kod konfiguruje też środowisko wyłączając ostrzeżenia przez `warnings.filterwarnings('ignore')`.

In [ ]:
class CFG:
    model = 'gpt-4o-mini'
    temp = 0

In [ ]:
os.environ["OPENAI_API_KEY"] = userdata.get('openaivision')
os.environ["SERPER_API_KEY"] = userdata.get('serper')

# Agenci

In [ ]:
llm = ChatOpenAI(model= CFG.model, temperature = CFG.temp)

Ten wiersz kodu tworzy instancję modelu językowego ChatOpenAI z dwoma parametrami pobranymi z obiektu konfiguracyjnego CFG:

- model - określa konkretny model ChatGPT, który ma być używany (nazwa modelu jest zapisana w CFG.model)
- temperature - ustawia parametr losowości odpowiedzi (wartość z CFG.temp)

Temperature to parametr w zakresie 0-1:
- Niskie wartości (bliskie 0) dają bardziej przewidywalne, spójne odpowiedzi
- Wysokie wartości (bliskie 1) zwiększają kreatywność i różnorodność odpowiedzi

Utworzona instancja llm będzie używana do generowania tekstu i komunikacji z API OpenAI.

In [ ]:
# load tools
serper_search_tool = SerperDevTool()
scrap_tool = ScrapeWebsiteTool()
website_search_tool = WebsiteSearchTool()

# define the tools to be used by the Agents
tools=[serper_search_tool, scrap_tool, website_search_tool]


Te linie kodu przygotowują zestaw narzędzi dla agentów AI:

Tworzą trzy narzędzia:
- serper_search_tool - narzędzie do wyszukiwania w internecie przez API Serper
- scrap_tool - narzędzie do wyciągania treści ze stron internetowych
- website_search_tool - narzędzie do przeszukiwania konkretnych stron

Następnie te trzy narzędzia są łączone w listę tools, która będzie przekazana agentom do wykorzystania podczas wykonywania zadań.

Każde z tych narzędzi rozszerza możliwości agentów w inny sposób:
- pierwsze pozwala im znajdować informacje w sieci
- drugie umożliwia pobieranie zawartości stron
- trzecie pomaga w przeszukiwaniu konkretnych witryn

In [ ]:
# agent definitions
travel_expert = Agent(
  role='Specjalista ds. Hoteli',
  goal='Sprawnie wyszukiwać i tworzyć kompleksową listę odpowiednich opcji hotelowych w określonej lokalizacji  \
  przestrzegając podanych kryteriów, takich jak daty zameldowania/wymeldowania, liczba gości i wszelkie inne szczególne wymagania. \
  Zapewnić, że zebrane informacje są dokładne, aktualne i zawierają kluczowe szczegóły, jak ceny, udogodnienia i oceny gości.',
  backstory='Jesteś doświadczonym cyfrowym concierge podróży z wyczuleniem na szczegóły i pasją do znajdowania idealnego zakwaterowania. \
  Dzięki wieloletniemu doświadczeniu w branży hotelarskiej i rozległej wiedzy o światowych sieciach hoteli oraz butikowych obiektach, \
  doskonalisz swoje umiejętności w poruszaniu się po różnych platformach rezerwacyjnych i bazach danych hoteli. \
  Twoja specjalizacja polega na szybkim przeglądaniu licznych opcji w celu identyfikacji najbardziej odpowiednich wyborów dla podróżnych, \
  biorąc pod uwagę czynniki takie jak lokalizacja, cena, udogodnienia i opinie gości.',
  verbose=True,
  allow_delegation=False,
  llm=llm,
  tools=tools,
)


hotel_reviewer = Agent(
  role='Analityk Jakości Hoteli',
  goal='Dokładnie oceniać i porównywać opcje hotelowe, aby dostarczać szczegółowych, bezstronnych ocen pomagających podróżnym w podejmowaniu świadomych decyzji. \
  Analizować udogodnienia, lokalizację, stosunek jakości do ceny i doświadczenia gości każdego hotelu, aby tworzyć kompleksowe recenzje podkreślające mocne strony, \
  słabości i unikalne cechy.',
  backstory = "Jesteś doświadczonym profesjonalistą branży turystycznej z ponad dziesięcioletnim doświadczeniem w ocenie i krytyce hoteli. \
  Twoje doświadczenie obejmuje pracę jako inspektor hoteli luksusowych, dziennikarz turystyczny dla znanych publikacji oraz konsultant systemów oceny hoteli.  \
  To różnorodne doświadczenie rozwinęło Twoją zdolność do oceny zakwaterowania z wielu perspektyw, uwzględniając zarówno obiektywne kryteria, \
  jak i subiektywne doświadczenia gości.   Masz szczególny talent do identyfikowania ukrytych perełek i dostrzegania potencjalnych problemów, które mogą wpłynąć na pobyt gościa.",
  verbose=True,
  allow_delegation=False,
  llm=llm,
  tools=tools,
)

Ten kod tworzy dwóch specjalizowanych agentów AI do obsługi zadań związanych z hotelami:

travel_expert to agent specjalizujący się w wyszukiwaniu hoteli:
- jego rola to "Specjalista ds. Hoteli"
- ma za zadanie wyszukiwać opcje hotelowe według określonych kryteriów
- posiada "backstory" jako doświadczony concierge z wiedzą o branży
- ma dostęp do wcześniej zdefiniowanych narzędzi (tools)
- pracuje z modelem językowym (llm)
- ma włączone szczegółowe logowanie (verbose=True)
- nie może delegować zadań (allow_delegation=False)

hotel_reviewer to agent oceniający hotele:
- pełni rolę "Analityka Jakości Hoteli"
- jego zadaniem jest szczegółowa analiza i porównywanie opcji hotelowych
- ma "backstory" jako ekspert z 10-letnim doświadczeniem w branży
- używa tych samych narzędzi i modelu co travel_expert
- również ma włączone logowanie i wyłączoną delegację zadań

Obaj agenci mają szczegółowo określone cele i historie, które wpływają na sposób, w jaki będą podchodzić do powierzonych im zadań.

In [ ]:
# tasks

hotel_search_task = Task(
      description = "Wyszukaj 5 hoteli w {location} dla {number_of_people} osób dorosłych, z zameldowaniem {check_in} i wymeldowaniem {check_out}.",
      agent = travel_expert,
      expected_output = "Wszystkie szczegóły dotyczące konkretnie wybranego zakwaterowania."
)



hotel_review_task = Task(
  description="Na podstawie dostarczonych rekomendacji, wybierz najlepsze opcje w oparciu o oceny, recenzje i dostępne udogodnienia. \
  Weź pod uwagę, że budżet wynosi {budget} PLN. Postaraj się znaleźć zakwaterowanie w granicach tego budżetu.",
  expected_output="Wszystkie szczegóły dotyczące konkretnie wybranego zakwaterowania, w tym cena, URL i ewentualne zdjęcia, jeśli są dostępne.",
  agent = hotel_reviewer,
)

Ten kod definiuje dwa zadania dla wcześniej utworzonych agentów:

hotel_search_task to zadanie wyszukiwania hoteli:
- przypisane do agenta travel_expert
- opis zawiera parametry, które będą uzupełniane przy uruchomieniu:
  - {location} - lokalizacja
  - {number_of_people} - liczba osób
  - {check_in} - data zameldowania
  - {check_out} - data wymeldowania
- oczekuje zwrotu szczegółowych informacji o znalezionych opcjach zakwaterowania

hotel_review_task to zadanie analizy i wyboru najlepszych opcji:
- przypisane do agenta hotel_reviewer
- uwzględnia parametr {budget} określający limit cenowy w PLN
- ma wybrać najlepsze opcje na podstawie:
  - ocen
  - recenzji
  - dostępnych udogodnień
- oczekuje szczegółowego raportu zawierającego:
  - cenę
  - link do oferty
  - dostępne zdjęcia

Te zadania tworzą dwuetapowy proces: najpierw wyszukanie opcji, potem ich szczegółowa analiza i wybór najlepszych propozycji.

In [ ]:
# Create the Crew
travel_agent_crew = Crew(
    agents=[travel_expert],
    tasks=[hotel_search_task],
    verbose=True
)

event_criteria = {
    'location': 'Lublin, Poland',
    'check_in': '22 grudnia 2024',
    'check_out': '27 grudnia 2024',
    'number_of_people': 4,
    'budget': 2000
}

result = travel_agent_crew.kickoff(inputs=event_criteria)

# Agent: Specjalista ds. Hoteli
## Task: Wyszukaj 5 hoteli w Lublin, Poland dla 4 osób dorosłych, z zameldowaniem 22 grudnia 2024 i wymeldowaniem 27 grudnia 2024.


# Agent: Specjalista ds. Hoteli
## Thought: Muszę wyszukać hotele w Lublinie, Polska, dla 4 osób dorosłych, z datami zameldowania 22 grudnia 2024 i wymeldowania 27 grudnia 2024. Rozpocznę od przeszukania internetu w celu znalezienia odpowiednich opcji hotelowych.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"hotele w Lublinie Polska dla 4 os\\u00f3b doros\\u0142ych 22 grudnia 2024 - 27 grudnia 2024\"}"
## Tool Output: 

Search results: Title: 10 najlepszych hoteli w mieście Lublin (ceny od 162 zł)
Link: https://www.booking.com/city/pl/lublin.pl.html
Snippet: Trzygwiazdkowy hotel Campanile Lublin usytuowany jest 1,4 km od Starego Miasta w Lublinie i oferuje klimatyzowane pokoje z łazienką z wanną lub prysznicem.
---
Title: Booking.com | Oficjalna strona | Najlepsze hotele, loty, samochody ...
Link:

Inserting batches in chromadb: 100%|██████████| 1/1 [00:01<00:00,  1.02s/it]




# Agent: Specjalista ds. Hoteli
## Thought: Muszę teraz przeszukać stronę Booking.com, aby znaleźć konkretne hotele w Lublinie, które spełniają podane kryteria.
## Using tool: Search in a specific website
## Tool Input: 
"{\"search_query\": \"hotele w Lublinie dla 4 os\\u00f3b doros\\u0142ych 22 grudnia 2024 - 27 grudnia 2024\", \"website\": \"https://www.booking.com/city/pl/lublin.pl.html\"}"
## Tool Output: 
Relevant Content:
10 najlepszych hoteli w mieście Lublin (ceny od US$42) Przejdź do głównej treści Wyszukaj hotele w mieście LublinPodaj daty, aby zobaczyć najnowsze ceny i oferty hoteli w mieście Lublin Pokaż hotele na mapieFiltruj według następujących kryteriów:Liczba gwiazdek5 gwiazdek4 gwiazdki3 gwiazdki2 gwiazdki1 gwiazdkaOcena GościZnakomity: ponad 9Bardzo dobry: ponad 8Dobry: ponad 7Przyjemny: ponad 6Lublin – 224 hotele i inne miejsca na pobytWybrane przez nasNajniższa cenaLiczba gwiazdek i cenaNajczęściej i najlepiej ocenianeZobacz aktualne ceny i oferty, wybierając swó

In [ ]:
ppr(result.raw)

('1. **Campanile Lublin**\n'
 '   - **Lokalizacja:** 1,4 km od Starego Miasta w Lublinie\n'
 '   - **Typ:** Hotel trzygwiazdkowy\n'
 '   - **Ceny:** od 44,78 USD za noc\n'
 '   - **Udogodnienia:** Klimatyzowane pokoje, łazienka z wanną lub '
 'prysznicem\n'
 '   - **Ocena gości:** 7.5/10 (Dobra lokalizacja, pozytywne opinie)\n'
 '\n'
 '2. **Focus Hotel Premium Lublin Conference & SPA**\n'
 '   - **Lokalizacja:** 8 km od centrum Lublina\n'
 '   - **Typ:** Hotel czterogwiazdkowy\n'
 '   - **Ceny:** od 80,35 USD za noc\n'
 '   - **Udogodnienia:** Spa, centrum konferencyjne\n'
 '   - **Ocena gości:** 9.1/10 (Znakomity, często polecany)\n'
 '\n'
 '3. **Hotel Agit Congress & Spa**\n'
 '   - **Lokalizacja:** Na przedmieściach Lublina\n'
 '   - **Typ:** Hotel czterogwiazdkowy\n'
 '   - **Ceny:** od 69,65 USD za noc\n'
 '   - **Udogodnienia:** Ekologiczne zakwaterowanie, zdrowe posiłki\n'
 '   - **Ocena gości:** 8.9/10 (Fantastyczny, wysoka jakość usług)\n'
 '\n'
 '4. **Hampton By Hilton Lublin